In [ ]:
pip cache purge

Files removed: 0


In [ ]:

!pip install transformers
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 4.4 MB/s eta 0:00:00


In [ ]:
# !pip install sentencepiece
# !wget https://raw.githubusercontent.com/google/sentencepiece/master/data/botchan.txt

In [ ]:
import torch
import pandas as pd
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount = True)
link = '/content/drive/MyDrive/content/myntra_products_catalog.csv'
data = pd.read_csv(link)  # myntra data

Mounted at /content/drive


In [ ]:
# Prepare source and target texts
source_texts = ["product title: " + title for title in data["ProductName"]]
target_texts = ["description: " + desc for desc in data["Description"]]

In [ ]:
# Load the BART tokenizer and model
model_name = "facebook/bart-base"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)


In [ ]:
# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=

In [ ]:
# Tokenize data
train_encodings = tokenizer(source_texts, truncation=True, padding='max_length', max_length=40)
train_labels = tokenizer(target_texts, truncation=True, padding='max_length', max_length=40).input_ids

# Prepare dataset for HuggingFace Seq2SeqTrainer
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(train_encodings, train_labels)

In [ ]:
!pip show transformers
!pip show accelerate


Name: transformers
Version: 4.32.1
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 
Name: accelerate
Version: 0.22.0
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: sylvain@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy, packaging, psutil, pyyaml, torch
Required-by: 


In [ ]:
# Define the training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./bart",
    per_device_train_batch_size=3,
    evaluation_strategy="no",
    eval_steps=100,  # Adjust as needed
    save_steps=500,  # Adjust as needed
    save_total_limit=2,
    num_train_epochs=1,  # Adjust as needed
)

# Create a Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=None,  # You can define a custom data collator if needed
    tokenizer=tokenizer,
)

In [ ]:
# Start fine-tuning
trainer.train()

# Save the trained model
trainer.save_model()



Step,Training Loss
500,1.755100
1000,0.928100
1500,0.771900
2000,0.716300
2500,0.606400
3000,0.607700
3500,0.551700
4000,0.546000


In [ ]:
# Load the fine-tuned model for generation
model = BartForConditionalGeneration.from_pretrained("./bart").to(device)

# Generate text using the fine-tuned model
input_text = "White T-shirt"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)
generated_ids = model.generate(input_ids,min_length = 70 ,max_length=100, num_beams=4, length_penalty=2.0, early_stopping=True)
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

In [ ]:
# Print the generated text
print("Generated Text:", generated_text)


Generated Text: description: White printed T-shirt, has a round neck, and short sleevesEngineered with Antistatix, Drytech+ and RapidFoam+ for high-impact andWarranty: 1 yearWarrOur stylist has paired this pair of trousers with a belt.This pairof trousers does not come with a


In [ ]:
# Load the fine-tuned model for generation
model = BartForConditionalGeneration.from_pretrained("./bart").to(device)

# Generate text using the fine-tuned model
input_text = "Black Shoes"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)
generated_ids = model.generate(input_ids,min_length = 100 ,max_length=150, num_beams=4, length_penalty=2.0, early_stopping=True)
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

In [ ]:
# Print the generated text
print("Generated Text:", generated_text)


Generated Text: description: A pair of black running sports shoes, has regular styling, lace-up detailSynthetic upperCushioned footbedTextured and patterned outsoleWarranty: 3 monthsWearing a beltWarrOur stylist has paired this pair of sports shoes with a belt.This pair of trousers does not come with the belt. This pair of shorts does notWarrange the belt with a tang closureComes with a stitched version of the blouseWarr


In [ ]:
# Generate text using the fine-tuned model
input_text = "Blue bag"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)
generated_ids = model.generate(input_ids,min_length = 100 ,max_length=150, num_beams=4, length_penalty=2.0, early_stopping=True)
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

In [ ]:
print("Generated Text:", generated_text)

Generated Text: Blue bag, has a zip closure1 main compartment, 1 external and 4 inner pocketsTablet sleeve: NoTwo Handles with a detachable sling strapWarranty: 1 yearWW
,,Key ,description gathers  flared comp concealedInt Zip coin pocketNon-Detachable SlingStrap style: Brown regular, leather strap with a tang closureNon-detachable sling straps
